In [ ]:
import sys

# change local PATH environment for Python
# sys.path.append('/nfs/optimi/usr/sw/cplex/python/3.6/x86-64_linux')

import cplex
import networkx as nx
import time
import re
from scipy import *
from scipy.sparse import *
import numpy as np
#import Logger  # print out to file
from datetime import datetime, timedelta
from dateutil.parser import parse
import matplotlib.pyplot as plt

from tools import *
from OD_matrix import *

# networkx start
graph = nx.DiGraph() # nx.MultiDiGraph()

inspectors = { 0 : {"base": 'RDRM', "working_hours": 8, "rate": 12},
              1 : {"base": 'HH', "working_hours": 5, "rate": 10},
              2 : {"base": 'AHAR', "working_hours": 6, "rate": 15}}

flow_var_names = []

# dictionary with keys being var_M and values being upper-bounds
var_passengers_inspected = {}
# new reformulation variable
# has length = # of types of passengers
# need to enumerate types of passengers
# keys are var_M and values are upper bounds which should all be 1
var_portion_of_passengers_inspected = np.array([])

HOUR_TO_SECONDS = 3600
MINUTE_TO_SECONDS = 60


input_dir = '../hai_code/Mon_Arcs.txt' # /home/optimi/bzfnguye/grips-2019

#============================= CONSTRUCTING THE GRAPH ============================================

print("Building graph ...", end = " ")
t1 = time.time()

# build_graph(input_dir, graph, inspectors, flow_var_names, var_passengers_inspected)

with open(input_dir, "r") as f:
    for line in f.readlines()[:-1]:
        line = line.replace('\n','').split(' ')
        start = line[0]+'@'+line[1]
        end = line[2]+'@'+line[3]

        for k in inspectors:
            flow_var_names.append('var_x_{}_{}_{}'.format(start, end, k))

        var_passengers_inspected['var_M_{}_{}'.format(start, end)] = int(line[4])

        graph.add_node(start, station = line[0], time_stamp = line[1])
        graph.add_node(end, station = line[2], time_stamp = line[3])

        # we assume a unique edge between events for now
        if not graph.has_edge(start, end):
            graph.add_edge(start, end, num_passengers= int(line[4]), travel_time =int(line[5]))


# time to build graph
t2 = time.time()

print('Finished! Took {:.5f} seconds'.format(t2-t1))

#================================ OD Estimation ===============================
print("Estimating OD Matrix ...", end = " ")

T, OD = generate_OD_matrix(graph)

# preliminary data needed
all_paths, path_idx = enumerate_all_shortest_paths(graph)

# create variable names
for (source, sink), value in all_paths.items():
    var_portion_of_passengers_inspected = np.append(var_portion_of_passengers_inspected, 'portion_of_({},{})'.format(source, sink))

t2a = time.time()
# print(new_weights)
print('Finished! Took {:.5f} seconds'.format(t2a-t2))

#============================== ADDING SOURCE/SINK NODES ==========================================

print("Adding Sinks/Sources...", end=" ")

# add_sources_and_sinks(graph, inspectors, flow_var_names, var_passengers_inspected)
for k, vals in inspectors.items():
    source = "source_" + str(k)
    sink = "sink_"+str(k)
    graph.add_node(source, station = vals['base'], time_stamp = None)
    graph.add_node(sink, station = vals['base'], time_stamp = None)
    for node in graph.nodes():
        if (graph.nodes[node]['station'] == vals['base']) and (graph.nodes[node]['time_stamp'] is not None):

            # adding edge between sink and events and adding to the variable dictionary
            graph.add_edge(source, node, num_passengers = 0, travel_time = 0)
            flow_var_names.append('var_x_{}_{}_{}'.format(source, node, k))
            var_passengers_inspected['var_M_{}_{}'.format(source, node)] = 0
            graph.add_edge(node, sink, num_passengers=0, travel_time = 0 )
            flow_var_names.append('var_x_{}_{}_{}'.format(node, sink, k))
            var_passengers_inspected['var_M_{}_{}'.format(node, sink)] = 0


t3 = time.time()

print('Finished! Took {:.5f} seconds'.format(t3-t2a))

# test edge source to sinks
# print('TEST: Unique edge between two nodes: ', num_edges == graph.number_of_edges())
print("TEST: No Source-Sink Edge: ", not graph.has_edge("source_0", "sink_0"))

# freeze graph to prevent further changes
graph = nx.freeze(graph)

print('successors of FFU@10:51:00')

for node in graph.successors('FFU@10:51:00'):
    print(node)

#================================== START CPLEX =================================================

print("Start CPLEX")

c = cplex.Cplex()

# start_cplex(c, flow_var_names, var_passengers_inspected)
# initialize_cplex(c, OD, flow_var_names, var_portion_of_passengers_inspected)
c.set_problem_type(c.problem_type.LP)
c.objective.set_sense(c.objective.sense.maximize)	# formulated as a maximization problem


#========================= ADDING VARIABLES AND OBJECTIVE FUNCTION ==============================

print("Adding variables...", end=" ")

# adding objective function and declaring variable types
c.variables.add(
    names = flow_var_names,
    types = [ c.variables.type.binary ] * len(flow_var_names)
)

# defining the objective function coefficients
obj = [OD[(source, sink)] for source, sink in OD.keys()]

In [ ]:
len(all_paths)

In [ ]:
c.variables.add(
    names = var_portion_of_passengers_inspected,
    lb = [0] * len(var_portion_of_passengers_inspected),
    ub = [1] * len(var_portion_of_passengers_inspected),
    obj = obj,
    types = [ c.variables.type.continuous ] * len(var_portion_of_passengers_inspected)
)

In [ ]:
len(obj)

In [ ]:
len(var_portion_of_passengers_inspected)

In [ ]:
len(OD)

In [ ]:
len(all_paths)

In [ ]:
OD.size()

In [ ]:
all_paths

In [ ]:
clear()

In [ ]:
all_paths

In [ ]:
quit()

In [ ]:
# Input: Data file in Forward Star format
# Output: Digraph object
# Function: This file will be used to read in information to
# create the DiGraph object and write it to another file
# @author: Ruby Abrams, Hai Nguyen

import sys

# change local PATH environment for Python
# sys.path.append('/nfs/optimi/usr/sw/cplex/python/3.6/x86-64_linux')

import cplex
import networkx as nx
import time
import re
from scipy import *
from scipy.sparse import *
import numpy as np
#import Logger  # print out to file
from datetime import datetime, timedelta
from dateutil.parser import parse
import matplotlib.pyplot as plt

from tools import *
from OD_matrix import *

# networkx start
graph = nx.DiGraph() # nx.MultiDiGraph()

inspectors = { 0 : {"base": 'RDRM', "working_hours": 8, "rate": 12},
              1 : {"base": 'HH', "working_hours": 5, "rate": 10},
              2 : {"base": 'AHAR', "working_hours": 6, "rate": 15}}

flow_var_names = []

# dictionary with keys being var_M and values being upper-bounds
var_passengers_inspected = {}
# new reformulation variable
# has length = # of types of passengers
# need to enumerate types of passengers
# keys are var_M and values are upper bounds which should all be 1
var_portion_of_passengers_inspected = np.array([])

HOUR_TO_SECONDS = 3600
MINUTE_TO_SECONDS = 60


input_dir = '../hai_code/Mon_Arcs.txt' # /home/optimi/bzfnguye/grips-2019

#============================= CONSTRUCTING THE GRAPH ============================================

print("Building graph ...", end = " ")
t1 = time.time()

# build_graph(input_dir, graph, inspectors, flow_var_names, var_passengers_inspected)

with open(input_dir, "r") as f:
    for line in f.readlines()[:-1]:
        line = line.replace('\n','').split(' ')
        start = line[0]+'@'+line[1]
        end = line[2]+'@'+line[3]

        for k in inspectors:
            flow_var_names.append('var_x_{}_{}_{}'.format(start, end, k))

        var_passengers_inspected['var_M_{}_{}'.format(start, end)] = int(line[4])

        graph.add_node(start, station = line[0], time_stamp = line[1])
        graph.add_node(end, station = line[2], time_stamp = line[3])

        # we assume a unique edge between events for now
        if not graph.has_edge(start, end):
            graph.add_edge(start, end, num_passengers= int(line[4]), travel_time =int(line[5]))


# time to build graph
t2 = time.time()

print('Finished! Took {:.5f} seconds'.format(t2-t1))

#================================ OD Estimation ===============================
print("Estimating OD Matrix ...", end = " ")

T, OD = generate_OD_matrix(graph)

# preliminary data needed
all_paths, path_idx = enumerate_all_shortest_paths(graph)

# create variable names
for (source, sink), value in all_paths.items():
    var_portion_of_passengers_inspected = np.append(var_portion_of_passengers_inspected, 'portion_of_({},{})'.format(source, sink))

t2a = time.time()
# print(new_weights)
print('Finished! Took {:.5f} seconds'.format(t2a-t2))

#============================== ADDING SOURCE/SINK NODES ==========================================

print("Adding Sinks/Sources...", end=" ")

# add_sources_and_sinks(graph, inspectors, flow_var_names, var_passengers_inspected)
for k, vals in inspectors.items():
    source = "source_" + str(k)
    sink = "sink_"+str(k)
    graph.add_node(source, station = vals['base'], time_stamp = None)
    graph.add_node(sink, station = vals['base'], time_stamp = None)
    for node in graph.nodes():
        if (graph.nodes[node]['station'] == vals['base']) and (graph.nodes[node]['time_stamp'] is not None):

            # adding edge between sink and events and adding to the variable dictionary
            graph.add_edge(source, node, num_passengers = 0, travel_time = 0)
            flow_var_names.append('var_x_{}_{}_{}'.format(source, node, k))
            var_passengers_inspected['var_M_{}_{}'.format(source, node)] = 0
            graph.add_edge(node, sink, num_passengers=0, travel_time = 0 )
            flow_var_names.append('var_x_{}_{}_{}'.format(node, sink, k))
            var_passengers_inspected['var_M_{}_{}'.format(node, sink)] = 0


t3 = time.time()

print('Finished! Took {:.5f} seconds'.format(t3-t2a))

# test edge source to sinks
# print('TEST: Unique edge between two nodes: ', num_edges == graph.number_of_edges())
print("TEST: No Source-Sink Edge: ", not graph.has_edge("source_0", "sink_0"))

# freeze graph to prevent further changes
graph = nx.freeze(graph)

print('successors of FFU@10:51:00')

for node in graph.successors('FFU@10:51:00'):
    print(node)

#================================== START CPLEX =================================================

print("Start CPLEX")

c = cplex.Cplex()

# start_cplex(c, flow_var_names, var_passengers_inspected)
# initialize_cplex(c, OD, flow_var_names, var_portion_of_passengers_inspected)
c.set_problem_type(c.problem_type.LP)
c.objective.set_sense(c.objective.sense.maximize)	# formulated as a maximization problem


#========================= ADDING VARIABLES AND OBJECTIVE FUNCTION ==============================

print("Adding variables...", end=" ")

# adding objective function and declaring variable types
c.variables.add(
    names = flow_var_names,
    types = [ c.variables.type.binary ] * len(flow_var_names)
)

# defining the objective function coefficients
obj = [OD[(source, sink)] for source, sink in OD.keys()]

In [ ]:
clc

In [4]:
# Input: Data file in Forward Star format
# Output: Digraph object
# Function: This file will be used to read in information to
# create the DiGraph object and write it to another file
# @author: Ruby Abrams, Hai Nguyen

import sys

# change local PATH environment for Python
# sys.path.append('/nfs/optimi/usr/sw/cplex/python/3.6/x86-64_linux')

import cplex
import networkx as nx
import time
import re
from scipy import *
from scipy.sparse import *
import numpy as np
#import Logger  # print out to file
from datetime import datetime, timedelta
from dateutil.parser import parse
import matplotlib.pyplot as plt

from tools import *
from OD_matrix import *

# networkx start
graph = nx.DiGraph() # nx.MultiDiGraph()

inspectors = { 0 : {"base": 'RDRM', "working_hours": 8, "rate": 12},
              1 : {"base": 'HH', "working_hours": 5, "rate": 10},
              2 : {"base": 'AHAR', "working_hours": 6, "rate": 15}}

flow_var_names = []

# dictionary with keys being var_M and values being upper-bounds
var_passengers_inspected = {}
# new reformulation variable
# has length = # of types of passengers
# need to enumerate types of passengers
# keys are var_M and values are upper bounds which should all be 1
var_portion_of_passengers_inspected = np.array([])

HOUR_TO_SECONDS = 3600
MINUTE_TO_SECONDS = 60


input_dir = '../hai_code/Mon_Arcs.txt' # /home/optimi/bzfnguye/grips-2019

#============================= CONSTRUCTING THE GRAPH ============================================

print("Building graph ...", end = " ")
t1 = time.time()

# build_graph(input_dir, graph, inspectors, flow_var_names, var_passengers_inspected)

with open(input_dir, "r") as f:
    for line in f.readlines()[:-1]:
        line = line.replace('\n','').split(' ')
        start = line[0]+'@'+line[1]
        end = line[2]+'@'+line[3]

        for k in inspectors:
            flow_var_names.append('var_x_{}_{}_{}'.format(start, end, k))

        var_passengers_inspected['var_M_{}_{}'.format(start, end)] = int(line[4])

        graph.add_node(start, station = line[0], time_stamp = line[1])
        graph.add_node(end, station = line[2], time_stamp = line[3])

        # we assume a unique edge between events for now
        if not graph.has_edge(start, end):
            graph.add_edge(start, end, num_passengers= int(line[4]), travel_time =int(line[5]))


# time to build graph
t2 = time.time()

print('Finished! Took {:.5f} seconds'.format(t2-t1))

#================================ OD Estimation ===============================
print("Estimating OD Matrix ...", end = " ")

T, OD = generate_OD_matrix(graph)

# preliminary data needed
all_paths, path_idx = enumerate_all_shortest_paths(graph)

# create variable names
for (source, sink), value in all_paths.items():
    var_portion_of_passengers_inspected = np.append(var_portion_of_passengers_inspected, 'portion_of_({},{})'.format(source, sink))

t2a = time.time()
# print(new_weights)
print('Finished! Took {:.5f} seconds'.format(t2a-t2))

#============================== ADDING SOURCE/SINK NODES ==========================================

print("Adding Sinks/Sources...", end=" ")

# add_sources_and_sinks(graph, inspectors, flow_var_names, var_passengers_inspected)
for k, vals in inspectors.items():
    source = "source_" + str(k)
    sink = "sink_"+str(k)
    graph.add_node(source, station = vals['base'], time_stamp = None)
    graph.add_node(sink, station = vals['base'], time_stamp = None)
    for node in graph.nodes():
        if (graph.nodes[node]['station'] == vals['base']) and (graph.nodes[node]['time_stamp'] is not None):

            # adding edge between sink and events and adding to the variable dictionary
            graph.add_edge(source, node, num_passengers = 0, travel_time = 0)
            flow_var_names.append('var_x_{}_{}_{}'.format(source, node, k))
            var_passengers_inspected['var_M_{}_{}'.format(source, node)] = 0
            graph.add_edge(node, sink, num_passengers=0, travel_time = 0 )
            flow_var_names.append('var_x_{}_{}_{}'.format(node, sink, k))
            var_passengers_inspected['var_M_{}_{}'.format(node, sink)] = 0


t3 = time.time()

print('Finished! Took {:.5f} seconds'.format(t3-t2a))

# test edge source to sinks
# print('TEST: Unique edge between two nodes: ', num_edges == graph.number_of_edges())
print("TEST: No Source-Sink Edge: ", not graph.has_edge("source_0", "sink_0"))

# freeze graph to prevent further changes
graph = nx.freeze(graph)

print('successors of FFU@10:51:00')

for node in graph.successors('FFU@10:51:00'):
    print(node)

#================================== START CPLEX =================================================

print("Start CPLEX")

c = cplex.Cplex()

# start_cplex(c, flow_var_names, var_passengers_inspected)
# initialize_cplex(c, OD, flow_var_names, var_portion_of_passengers_inspected)
c.set_problem_type(c.problem_type.LP)
c.objective.set_sense(c.objective.sense.maximize)	# formulated as a maximization problem


#========================= ADDING VARIABLES AND OBJECTIVE FUNCTION ==============================

print("Adding variables...", end=" ")

# adding objective function and declaring variable types
c.variables.add(
    names = flow_var_names,
    types = [ c.variables.type.binary ] * len(flow_var_names)
)

# defining the objective function coefficients
obj = [OD[(source, sink)] for source, sink in OD.keys()]
c.variables.add(
    names = var_portion_of_passengers_inspected,
    lb = [0] * len(var_portion_of_passengers_inspected),
    ub = [1] * len(var_portion_of_passengers_inspected),
    obj = obj,
    types = [ c.variables.type.continuous ] * len(var_portion_of_passengers_inspectedion_of_passengers_inspected)
)


Building graph ... Finished! Took 0.24423 seconds
Estimating OD Matrix ... 147536
147536
Finished! Took 29.85100 seconds
Adding Sinks/Sources... Finished! Took 0.03170 seconds
TEST: No Source-Sink Edge:  True
successors of FFU@10:51:00
FKHM@11:04:00
Start CPLEX
Adding variables... 

range(25007, 35587)

In [2]:
len(all_paths)

10580

In [3]:
len(OD)

10580

In [6]:
all_paths[('RW@20:26:00', 'RF@20:53:00')]

['RW@20:26:00', 'RF@20:53:00']

In [ ]:
for (u,v), path in all_paths.items():
    